In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Reading District info dataset
district = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
display(district.isnull().sum())
print(district.shape)

In [ ]:
# Making a copy
district_df1=district.copy()
# Fill NAN values in state column to perform groupby
district_df1['state'].fillna('NIL',inplace=True)
district_df1.head()

In [ ]:
# Groupby state
district_df1=district_df1.groupby('state')
# Getting all the NIL district number
district_df1=district_df1.get_group('NIL')
district_df1=district_df1['district_id']
# converting all the district id to str
district_df1=district_df1.astype('str')
district_df1

In [ ]:
# converting the series to list
district_nan_lst=district_df1.to_list()
print(district_nan_lst)
# allocating a filepath
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data/'
# allocating a filetype
filetype='.csv'
# concatenate strings of full filepath
district_nan_lst=[path+filename+filetype for filename in district_nan_lst]
district_nan_lst

In [ ]:
# Reading the files without the filepath listed above
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    if filename not in district_nan_lst:
        df = pd.read_csv(filename, index_col=None, header=0)
        district_id = filename.split("/")[4].split(".")[0]
        df["district_id"] = district_id
        li.append(df)
    
engagement_df = pd.concat(li)
engagement_df = engagement_df.reset_index(drop=True)
engagement_df.head()

In [ ]:
# print(engagement_df.loc[engagement_df['district_id']=='2685'])

In [ ]:
#Check null value of data
display(engagement_df.isnull().sum())
# make a copy
engage_df=engagement_df.copy()
# Drop unneccessary columns first
engage_df.drop(['lp_id','engagement_index','district_id'],axis=1,inplace=True)
# Drop NAN rows for pct_access
engage_df.dropna(axis=0,inplace=True)

In [ ]:
# display(engage_df.isnull().sum())

In [ ]:
# Creating another column to extract out the month from the given date
engage_df['month'] = engage_df['time'].str[5:7] # Slicing out the month

# Writing Function to replace month to 3 character month
def replace_month(data,month):
    month_dict={
               "01":"Jan",
               "02":"Feb",
               "03":"Mar",
               "04":"Apr",
               "05":"May",
               "06":"Jun",
               "07":"Jul",
               "08":"Aug",
               "09":"Sep",
               "10":"Oct",
               "11":"Nov",
               "12":"Dec"}
    
    for key,value in month_dict.items():
        if month == key:
            data.replace(to_replace=month,value=value,inplace=True)
    return data

# Replacing the month value
lst=["01","02","03","04","05","06","07","08","09","10","11","12"]

for month in lst:
    replace_month(engage_df,month)
    
engage_df


In [ ]:
# Daily mean and median
df1=engage_df.copy()
df1=df1.groupby(['time']).mean()
df2=engage_df.copy()
df2=df2.groupby(['time']).median()
print(df1.head())
print(df2.head())

#Monthly mean and median
df3=engage_df.copy()
df3=df3.groupby(['month'],sort=False).mean()
df4=engage_df.copy()
df4=df4.groupby(['month'],sort=False).median()
print(df3.head())
print(df4.head())

In [ ]:
# Plotting Graphs that have removed the State with Missing Values From District Info Dataset
fig,axes = plt.subplots(2,2,figsize=(16,10))
sns.lineplot(ax=axes[0,0],data=df1,x='time',y='pct_access').xaxis.set_ticks([]) # daily mean
sns.lineplot(ax=axes[0,1],data=df2,x='time',y='pct_access').xaxis.set_ticks([]) # daily median
sns.lineplot(ax=axes[1,0],data=df3,x='month',y='pct_access') # monthly mean
sns.lineplot(ax=axes[1,1],data=df4,x='month',y='pct_access') # monthly median
axes[0,0].set_title("Daily Mean")
axes[0,1].set_title("Daily Median")
axes[1,0].set_title("Monthly Mean")
axes[1,1].set_title("Monthly Median")

The graphs above depict similar trend as the graphs that were plotted previously. This suggests that the data from the State with missing values in the district_info.csv did not affect the general trend observed. The reason for the sudden increase around the period of starting from July 2020 and August 2020 is still unknown. Perhaps, a closer look at the data at the State level instead of the National level may provide us some further insights in explaining the observation.